## Importing Libraries

In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## Importing Data

In [12]:
# main folder
path = r'C:\Users\steve\Documents\11.24 Instacart Basket Analysis'

# ords_checked
opc = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'opc_checked.pkl'))

## Analysis

#### 01. Regions

In [15]:
# create lists for each region
northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [16]:
# define function that will assign region based on [state] column
result = []

for value in opc['state']:
    if value in northeast:
        result.append('northeast')
    elif value in midwest:
        result.append('midwest')
    elif value in south:
        result.append('south')
    elif value in west:
        result.append('west')
    else: 
        result.append('error')

In [17]:
# set a new column in df equal to result
opc['region'] = result

In [18]:
opc['region'].value_counts(dropna = False)

region
south        10790096
west          8291679
midwest       7596065
northeast     5721892
Name: count, dtype: int64

In [19]:
# the following were used to troubleshoot when had values appearing with the 'error' tag
# ----------
#region_error = opc[opc['region'] == 'error']
#region_error['state'].value_counts(dropna = False)

#### 02. Spending Habits by Region

In [21]:
# show the counts of spending_flag for each region
spender_count_by_region = opc.groupby(['region'])['spending_flag'].agg(pd.Series.count)
spender_count_by_region

region
midwest       7596065
northeast     5721892
south        10790096
west          8291679
Name: spending_flag, dtype: int64

In [22]:
# show the counts of each value with spending_flag separated by region
spender_flag_count = opc.groupby(['region'])['spending_flag'].value_counts(dropna = False)

In [23]:
spender_flag_count

region     spending_flag
midwest    Low spender       7566800
           High spender        29265
northeast  Low spender       5703253
           High spender        18639
south      Low spender      10749519
           High spender        40577
west       Low spender       8260441
           High spender        31238
Name: count, dtype: int64

--------------------------------------------------------------
##### **Insight:** *All regions top spender type is 'low spender'*

--------------------------------------------------------------

In [25]:
# convert spender_flag counts into percentages
spend_percent = spender_flag_count.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).round(2).reset_index(level=0, drop=True)
# spend_percent

In [26]:
# convert series to a dataframe
spend_percent_df = spend_percent.to_frame()

# assign column name 'percent'
spend_percent_df.rename(columns = {'count': 'percent'}, inplace = True)
spend_percent_df

percent
region    spending_flag         
midwest   Low spender      99.61
          High spender      0.39
northeast Low spender      99.67
          High spender      0.33
south     Low spender      99.62
          High spender      0.38
west      Low spender      99.62
          High spender      0.38

--------------------------------------------------------------
##### **Insight:** *All regions have about 99.6 % low spenders and about .4% high spenders.*

--------------------------------------------------------------

#### 03. Exlusion flag for low activity customers

- **Low activity customers** = customers with < 5 orders

In [30]:
pd.options.display.max_columns = 35
opc.head(1)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,day_busyness,busiest_days,hour_of_day_busyness,max_order,loyalty_flag,avg_spent,spending_flag,med_days_since_prior_order,frequency_label,gender,state,age,date_joined,dependents,fam_status,income,region
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,south


In [31]:
# create a new df that stores all customers with a max order number less than 5
low_activity_cust = opc[opc['max_order'] < 5]
# show the count of customers per order number less than 5
low_activity_cust['max_order'].value_counts(dropna = False)

max_order
4    753414
3    686620
2         6
1         5
Name: count, dtype: int64

In [32]:
opc.shape

(32399732, 31)

In [33]:
low_activity_cust.shape

(1440045, 31)

In [34]:
# df excluding low activity should be 30959687 rows

# create a df that excludes these low activity customers
opc_minus_low_activity_cust = opc[opc['max_order'] >= 5]
opc_minus_low_activity_cust.shape

(30959687, 31)

#### 04. Customer Profiles

**Age**

In [37]:
# for simplicity, rename df
df = opc_minus_low_activity_cust

# find the age range of df using min and max
df['age'].min()

18

In [38]:
df['age'].max()

81

Age categories
- **young adult**: age < 25
- **adult**: age >= 25 and < 40
- **middle-aged**: age >= 40 and < 60
- **senior**: age >= 60

In [40]:
# define the function that labels ros based on the age group conditions
def age_label(row):
    if row['age'] < 25:
        return 'young adult'
    elif (row['age'] >= 25) and (row['age'] < 40):
        return 'adult'
    elif (row['age'] >= 40) and (row['age'] < 60):
        return 'middle-aged'
    elif row['age'] >= 60:
        return 'senior'
    else: return 'error'

In [41]:
# create new row for age group using the age_label function
df['age_group'] = df.apply(age_label, axis = 1)

C:\Users\steve\AppData\Local\Temp\ipykernel_10872\1092058153.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_group'] = df.apply(age_label, axis = 1)


In [42]:
# see the counts for each age range
df['age_group'].value_counts(dropna = False)

age_group
senior         10572857
middle-aged     9723390
adult           7248839
young adult     3414601
Name: count, dtype: int64

Income categories

In [44]:
df['income'].min()

25903

In [45]:
df['income'].max()

593901

Income categories
- **lower class**: income < 53,740
- **middle class**: income >= 53, 740 and <= 161,220 (cnbc definition of middle class in 2023)
- **upper class**: income > 161,220

In [47]:
# define the function that labels rows based on the income group conditions
def income_label(row):
    if row['income'] < 53740:
        return 'lower class'
    elif (row['income'] >= 53740) and (row['income'] <= 161220):
        return 'middle class'
    elif row['income'] > 161220:
        return 'upper class'
    else: return 'error'

In [48]:
# create new row for income group using the income_label function
df['income_group'] = df.apply(income_label, axis = 1)

C:\Users\steve\AppData\Local\Temp\ipykernel_10872\3804203610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['income_group'] = df.apply(income_label, axis = 1)


In [49]:
# see the counts for each age range
df['income_group'].value_counts(dropna = False)

income_group
middle class    24933489
lower class      4120189
upper class      1906009
Name: count, dtype: int64

Number of Dependents

Dependents flag:
- **No dependents**: dependents = 0 
- **Has dependents**: dependents > 0

In [52]:
# define the function that labels rows based on the income group conditions
def dependents_label(row):
    if row['dependents'] == 0:
        return 'No dependents'
    elif row['dependents'] > 0:
        return 'Has dependents'
    else: return 'error'

In [53]:
# create new row for age group using the age_label function
df['dependents_flag'] = df.apply(dependents_label, axis = 1)

C:\Users\steve\AppData\Local\Temp\ipykernel_10872\2848016000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dependents_flag'] = df.apply(dependents_label, axis = 1)


In [54]:
# see the counts for each age range
df['dependents_flag'].value_counts(dropna = False)

dependents_flag
Has dependents    23221179
No dependents      7738508
Name: count, dtype: int64

## Export Data

In [56]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))